Pigments from sPACE

PACE's OCI instrument has novel hyperspectral capabilities. Historically, distinguishing between species, functional groups, and pigments was difficult due to the multi-spectral nature of the available instruments, which collected spectra, but with gaps. PACE fills these gaps and provides complete spectra that allows for the distinguishment between oceanic bio-optical parameters.

There are many pigments involved in photosynthesis. The pigments that are utilized by different phytoplankton functional groups and species relates to their biology and ecology where different pigments are more utilized for certain niches than others. These pigments have unique spectral signatures that can be captured and differentiated spatially. 

Tremendous credit to Ali Chase as well as Emmanuel Boss, Ivona Cetinic, Wayne Slade, and Max Danenhower who created a Gaussian pigment estimation method that pulls salinity data from SMAP and temperature data from GHRSST, and uses it with PACE rrs data to estimate the concentration of chlorophyll pigments a, b, and c, as well as the concentration of photoprotective carotenoids for each pixel of PACE data. This was averaged for segments of three days. The data was then plotted where each pixel was represented by the concentration of the dominant pigment from the three-day average. Additionally, the bloom region indivudal pigment concentrations were averaged over the area to create bar plots.

In [ ]:
# Create earthaccess connection
import earthaccess
auth = earthaccess.login(persist=True)

In [ ]:
# Import your packages
# Some require pip install even if working in the CryoCloud
import utilspackagepace as utils
import rrs_inversion_pigments
import xarray
import numpy
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

In [ ]:
# Downloading rrs data from PACE (L3m, daily), salinity data from SMAP (L3, 8days), and temperature data from GHRSST (L4)
# Can customize temporal span and spatial resolution (options are 1deg, 0p1deg, 4km)
# Creates rrs_paths, sal_paths, and temp_paths folders in the operating folder to pull data in
rrs_paths, sal_paths, temp_paths = utils.load_data(('2024-07-07', '2024-07-09'), '4km')

In [ ]:
# Reads file names from PosixPath for each of the PACE images that fit the filters above and were downloaded
rrs_paths

In [ ]:
# Reads file names from PosixPath for each of the SMAP images that fit the filters above and were downloaded
# Some downlaoded before/after the date range
sal_paths

In [ ]:
# Reads file names from PosixPath for each of the GHRSST images that fit the filters above and were downloaded
# Some downlaoded before/after the date range
temp_paths

In [ ]:
# Creating an xarray dataset from rrs, sal, and temp files of the average values over the time period
# Define own coordinate region of ##N, ##S, ###W, ###E
avg_dataset = utils.create_dataset(rrs_paths, sal_paths, temp_paths, 28.5, 25.5, -160, -157)
# Reads the dataset
avg_dataset

In [ ]:
# Puts average values through algorithm
# Have to create a "pigments" folder in the operating folder for data to be deposited
# Also requires resources folder with Sullivan csv look-up table
pigments = utils.calculate_pigments(avg_dataset)
pigments

In [ ]:
# Creates a netcdf file of averaged pigment data
# Puts file in "pigments" folder (needs to be created)
# Enumerate "wash_coast" if completing series more than once
pigments.to_netcdf('pigments/wash_coast')

In [ ]:
# Pigments data becomes referenced as an xarray dataset for use
# Reads dataset
pigments = xarray.open_dataset('pigments/wash_coast')
pigments

In [ ]:
# PLots concentration of chlorophyll (a, b, c) or photoprotective carotenoids on a color map
# Can be repeated with any chlorophyll pigment or for PPC by using 'ppc'
chla = pigments['chla']
utils.plot_pigments(pigments['chla'], 0, 0.08, 'chla (mg/m^3)')

In [ ]:
# Mapping all of the pigments on one plot
# Each pixel will represent the concentration of the pigment with the highest average concentration relative to the two other pigments
# Continued in following cells

# Creating masks for each pigment dominance
# The masks themselves will be plotted so that pixels where that pigment average concentration is lower than that of the other two, it will be masked out
# The individual pigments will only be shown in pixels where its average is highest
# Works for three pigment groups
# PPC was excluded as most phytoplankton have photoprotective carotenoids, and does not necessarily represent the diversity that can be captured from hyperspectral data
mask_chla = (pigments['chla'] > pigments['chlb']) & (pigments['chla'] > pigments['chlc'])
mask_chlb = (pigments['chlb'] >= pigments['chla']) & (pigments['chlb'] > pigments['chlc'])
mask_chlc = (pigments['chlc'] >= pigments['chla']) & (pigments['chlc'] > pigments['chlb'])

In [ ]:
# Creating colormaps for each pigment
# Allows for display of concentration gradient for each pigment on the same plot
cmap_chla = plt.cm.Greens # Green gradient for chla
cmap_chlb = plt.cm.Reds # Red gradient for chlb
cmap_chlc = plt.cm.Blues # Blue gradient for chlc

In [ ]:
# Creating a plot for the data
# Not sure if this is needed so double check ***
fig, ax = plt.subplots(figsize=(8, 4))

In [ ]:
# Pulling in data to the plot as an application of the mask for all three pigments
# Applies colormap
# Adds colorbar (if added separately, it is improperly formatted)
# Sets vmin and vmax to create the same concentration scale for each pigment (0, 0.08) for accurate comparison
chla_plot = pigments['chla'].where(mask_chla).plot.imshow(cmap=cmap_chla, add_colorbar=True, ax=ax, vmin(0), vmax=(0.08))
chlb_plot = pigments['chlb'].where(mask_chlb).plot.imshow(cmap=cmap_chlb, add_colorbar=True, ax=ax, vmin(0), vmax=(0.08))
chlc_plot = pigments['chlc'].where(mask_chlc_.plot.imshow(cmap=cmap_chlc, add_colorbar=True, ax=ax, vmin(0), vmax=(0.08))

In [ ]:
# Format the plot
# Show the plot
plt.title('Dominant Pigment Concentration')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
# The following creates a bar plot of the chla, chlb, and chlc concentrations 
# for the time and space specified

# Defining variables to do the averages from the .nc 
# printing Individual pigment averages
# Printing string averages
stats = xarray.open_dataset('pigments/wash_coast')
statvariableA = stats['chla']
statvariableB = stats['chlb']
statvariableC = stats['chlc']
stataverageA = statvariableA.mean().item()
stataverageB = statvariableB.mean().item()
stataverageC = statvariableC.mean().item()
print(stataverageA)
print(stataverageB)
print(stataverageC)
stataverages = [stataverageA, stataverageB, stataverageC]
print(stataverages)

In [ ]:
# Creating the bar plot and formatting
labels = ['chla average', 'chlb average', 'chlc average']
plt.figure(figsize=(8, 6))
plt.bar(labels, july28averages, color=['green', 'red', 'blue'])
plt.title('Bar Plot of chla, chlb, and chlc Averages for July 28-30 in Bloom Region')
plt.xlabel('Pigments')
plt.ylabel('Average Values (mg/m^3)')
plt.ylim(0, 0.05)
plt.show()